# Stage 5 (3.2 data_preprocessing_for_2_stage_model)

# Импортируем библиотеки

In [1]:
import warnings


# ----------------
# Data processing
# ----------------
import dill

import numpy as np
import pandas as pd


from tqdm.auto import tqdm


warnings.filterwarnings("ignore")

RANDOM_STATE = 42

# Импортируем пути

In [2]:
data_path = "../data_closed/"

In [3]:
models_path = "../models/"

In [4]:
candidates_data_path = models_path + "candidates_data/"

# Feature Engineering

In [5]:
# Загружаем таблицу данных для моделей первого уровня
with (
    # Пользователи из test_df, которым будут выданы
    # таргетирвонные рекомондации
    open(data_path + "bNr2t_users.dill", "rb") as users_f,
    # Загружаем таблицу данных для моделей первого уровня
    open(data_path + "base_models_data.dill", "rb") as base_f,
    # Загружаем таблицу данных для ранкера
    open(data_path + "ranker_data.dill", "rb") as ranker_f,
    # Загружаем таблицу айтемов
    open(data_path + "df_items_mod.dill", "rb") as items_f,
):

    bNr2t_users = dill.load(users_f)

    # Создадим датасет взаимодействий
    ranker_data_bNr = pd.concat(
        [
            dill.load(base_f)[
                ["user_id", "item_id", "dt", "ui_inter", "u_total_inter"]
            ],
            dill.load(ranker_f)[
                ["user_id", "item_id", "dt", "ui_inter", "u_total_inter"]
            ],
        ],
        axis=0,
    ).rename(
        columns={
            # переименуем для удобства
            "u_total_inter": "user_hist",
        }
    )
    # Так как импортируем таблицу с этапа 2.1: 
    # в ней категориальные фитчи уже закодированы и рассчитаны значения
    # для колонок "item_pop", "item_avg_hist"
    # Сейчас данные колонки необходимо перерасчитать
    df_items = dill.load(items_f).drop(columns=["item_pop", "item_avg_hist"])

In [ ]:
# Получаем популярность контента
ranker_data_bNr["item_pop"] = ranker_data_bNr.groupby("item_id")["user_id"].transform(
    "count"
)
# Получаем среднюю популярность контента, просматриваемого этим юзером
ranker_data_bNr["user_avg_pop"] = ranker_data_bNr.groupby("user_id")[
    "item_pop"
].transform("mean")

# Получаем среднюю длину истории пользователя, которые смотрит этот контент
ranker_data_bNr["item_avg_hist"] = ranker_data_bNr.groupby("item_id")[
    "user_hist"
].transform("mean")

# Получаем популярность последнего просмотренного контента
ranker_data_bNr.sort_values(
    by=["user_id", "dt"],
    ascending=[True, False],
    ignore_index=True,
    inplace=True,
)
ranker_data_bNr["user_last_pop"] = ranker_data_bNr.groupby("user_id")[
    "item_pop"
].transform("first")


# Clear unnessessary data for users
ranker_data_bNr = ranker_data_bNr[ranker_data_bNr["user_id"].isin(bNr2t_users)]

ranker_data_bNr

In [7]:
# Добавляем новые фичи в соответствующие таблицы
df_items = pd.merge(
    left=df_items,
    right=(
        ranker_data_bNr[["item_id", "item_pop", "item_avg_hist"]].drop_duplicates()
    ),
    how="left",
    on="item_id",
)

# Создаем таблицу с фитчами пользователей
df_users = ranker_data_bNr[
    ["user_id", "user_hist", "user_avg_pop", "user_last_pop"]
].drop_duplicates()

In [8]:
# Save updated tables

with open(data_path + "df_items_mod_bNr.dill", "wb") as f:
    dill.dump(df_items, f)

with open(data_path + "df_users_bNr.dill", "wb") as f:
    dill.dump(df_users, f)

# Модель второго уровня (ранкер)

## Ranker Data

### Load

In [9]:
# Загружаем таблицу данных для моделей первого уровня
with (
    # Пользователи из test_df, которым будут выданы
    # таргетирвонные рекомондации
    open(data_path + "bNr2t_users.dill", "rb") as users_f,
    # Загружаем таблицу данных для моделей первого уровня
    open(data_path + "base_models_data.dill", "rb") as base_f,
    # Загружаем таблицу данных для ранкера
    open(data_path + "ranker_data.dill", "rb") as ranker_f,
    # Загружаем таблицу кандидатов
    open(candidates_data_path + "candidates_bNr_full.dill", "rb") as candidates_f,
):

    bNr2t_users = dill.load(users_f)

    # Создадим датасет взаимодействий
    ranker_data_bNr = pd.concat(
        [
            dill.load(base_f)[["user_id", "item_id", "ui_inter"]],
            dill.load(ranker_f)[["user_id", "item_id", "ui_inter"]],
        ],
        axis=0,
    )

    # Clear unnessessary data for users
    ranker_data_bNr = ranker_data_bNr[ranker_data_bNr["user_id"].isin(bNr2t_users)]

    # Загружаем таблицу кандидатов
    candidates_full = dill.load(candidates_f)

In [10]:
default_values_candidates = {
    "cos_score": candidates_full["cos_score"].min(),
    "bm25_score": candidates_full["bm25_score"].min(),
    "tfidf_score": candidates_full["tfidf_score"].min(),
    "lfm_score": candidates_full["lfm_score"].min(),
    "cos_rank": candidates_full["cos_rank"].max(),
    "bm25_rank": candidates_full["bm25_rank"].max(),
    "tfidf_rank": candidates_full["tfidf_rank"].max(),
    "lfm_rank": candidates_full["lfm_rank"].max(),
}

### Test data

In [11]:
# Оставляем среди users только тех, для кого есть 
# и рекомендации и таргеты
def users_filter(
    user_list: np.ndarray,
    candidates_df: pd.DataFrame,
    df: pd.DataFrame,
) -> pd.DataFrame:
    """
    Filters user interaction data and candidate recommendations, 
    ensuring each user has both interactions and recommendations.

    Args:
        user_list (np.ndarray): User IDs to include.
        candidates_df (pd.DataFrame): Candidate item recommendations 
            with ranks ('cos_rank', 'bm25_rank', 'lfm_rank', 'tfidf_rank').
        df (pd.DataFrame): User-item interactions ('user_id', 'item_id', 'dt', 
            and potentially other weight-based columns).

    Returns:
        pd.DataFrame: Filtered and merged DataFrame with user interactions 
            and candidate items sorted and with missing values filled. 
            It also filters down to items with at least one rank < 15
    """
    # For fillna
    default_values = {
        "ui_inter": 0,
        **default_values_candidates,
    }

    # Get valid interactions
    df = df[df["user_id"].isin(user_list)]
    candidates_df = candidates_df[candidates_df["user_id"].isin(user_list)]

    # join interaction на наших кандидатов для users из train, val, test
    df = df.merge(
        candidates_df,
        how="outer",
        on=["user_id", "item_id"],
    )

    df.fillna(default_values, inplace=True)
    df["ui_inter"] = df["ui_inter"].astype(int)

    # Сортируем по user_id
    df.sort_values(
        by=["user_id", "item_id"],
        inplace=True,
    )
    
    return df[
        (df["cos_rank"] < 15)
        | (df["bm25_rank"] < 15)
        | (df["lfm_rank"] < 15)
        | (df["tfidf_rank"] < 15)
    ]


In [ ]:
ranker_test = users_filter(bNr2t_users, candidates_full, ranker_data_bNr)
ranker_test

In [13]:
# Save
with open(data_path + "ranker_test_bNr.dill", "wb") as f:
    dill.dump(ranker_test, f)

## Добавим фитчи предметов и пользователей 

### Пользователей

In [14]:
# Загружаем таблицу фитчей пользователей
with open(data_path + "df_users_bNr.dill", "rb") as f:
    df_users = dill.load(f)

# Для новых фичей юзеров
default_values_users = {
    "user_hist": 0,
    "user_avg_pop": df_users["user_avg_pop"].median(),
    "user_last_pop": df_users["user_last_pop"].median(),
}

In [15]:
# Добавляем фичи
def add_users_features(
    df: pd.DataFrame,
    users: pd.DataFrame,
) -> pd.DataFrame:
    """
    Merges user and item features into a DataFrame, handling missing values.

    Args:
        df (pd.DataFrame): Interaction DataFrame ('user_id', 'item_id').
        users (pd.DataFrame): User features DataFrame ('user_id').
        items (pd.DataFrame): Item features DataFrame ('item_id').

    Returns:
        pd.DataFrame: DataFrame with merged user and item features, 
            and missing values filled.
    """
    users = users[users["user_id"].isin(df["user_id"])]
    df = pd.merge(df, users, how="left", on=["user_id"])

    # При джойне могут получиться строки
    # с несуществующими айтемами или юзерами.
    # Заполняем пропуски
    df.fillna(default_values_users, inplace=True)

    return df

In [16]:
# Загрузим таблицу ranker_test
with open(data_path + "ranker_test_bNr.dill", "rb") as f:
    ranker_test = dill.load(f)

ranker_test = add_users_features(ranker_test, df_users)

# Save
with open(data_path + "ranker_test_bNr.dill", "wb") as f:
    dill.dump(ranker_test, f)

### Предметов

In [5]:
# Загружаем таблицу айтемов
with (
    open(data_path + "df_items_mod_bNr.dill", "rb") as items_f,
):
    df_items = dill.load(items_f)


# Для новых фичей айтемов
default_values_items = {
    "item_pop": df_items["item_pop"].median(),
    "item_avg_hist": df_items["item_avg_hist"].median(),
}

In [6]:
# Добавляем фичи
def add_items_features(
    df: pd.DataFrame,
    items: pd.DataFrame,
) -> pd.DataFrame:
    """
    Merges user and item features into a DataFrame, handling missing values.

    Args:
        df (pd.DataFrame): Interaction DataFrame ('user_id', 'item_id').
        items (pd.DataFrame): Item features DataFrame ('item_id').

    Returns:
        pd.DataFrame: DataFrame with merged user and item features,
            and missing values filled.
    """

    items = items[items["item_id"].isin(df["item_id"].unique())]
    df = pd.merge(df, items, how="left", on=["item_id"])

    # # При джойне могут получиться строки
    # # с несуществующими айтемами или юзерами.
    # # Заполняем пропуски
    df.fillna(default_values_items, inplace=True)

    return df

In [7]:
# Загрузим таблицу ranker_test
with (
    open(data_path + "ranker_test_bNr.dill", "rb") as ranker_f,
):
    ranker_test = dill.load(ranker_f)

In [8]:
batches = np.array_split(ranker_test["user_id"].unique(), 100)

for i in [0, 20, 40, 60, 80]:
    res_table = []

    for batch in tqdm(batches[i : i + 20]):
        res_table.append(
            add_items_features(ranker_test[ranker_test["user_id"].isin(batch)], df_items)
        )

    res_table = pd.concat(
        res_table,
        axis=0,
    )

    # Save
    with open(data_path + f"ranker_test_bNr_{i}.dill", "wb") as f:
        dill.dump(res_table, f)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

## Добавим таргет

In [9]:
def add_target(df: pd.DataFrame) -> pd.DataFrame:

    df["target"] = np.where(df["ui_inter"] > 1, 2, 1)
    df["target"] = np.where(df["ui_inter"] > 2, 4, df["target"])
    df["target"] = np.where(df["ui_inter"] > 4, 8, df["target"])
    df["target"] = np.where(df["ui_inter"] > 6, 10, df["target"])
    df["target"] = df["target"].astype(int)

    return df

In [10]:
for i in tqdm([0, 20, 40, 60, 80]):

    # Load
    with open(data_path + f"ranker_test_bNr_{i}.dill", "rb") as f:
        ranker_test = dill.load(f)

    # Add
    ranker_test = add_target(ranker_test)
    
    # Save
    with open(data_path + f"ranker_test_bNr_{i}.dill", "wb") as f:
        dill.dump(ranker_test, f)

  0%|          | 0/5 [00:00<?, ?it/s]